In [1]:
% load_ext autoreload
% autoreload 2


In [2]:
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/home/Jiahui/mob2crime'

In [3]:
import pandas as pd
import numpy as np
import geopandas as gp
import folium
from scipy.spatial import Voronoi, voronoi_plot_2d

from src.utils.gis import *
from src.utils.map_vis import *
import src.mex_helper as mex

# Using mex_helper

In [11]:
mex_tower = mex.tower()
mex_cities = mex.cities()
mex_tvor = mex.tower_vor()
mex_t2city = mex.tower_vor('cities',intersection_only=True)
mex_t2city_false = mex.tower_vor('cities',intersection_only=False)
mex_t2city= mex_t2city.merge(mex_t2city_false[['gtid','iarea', 'gtid_area','weight']], on='gtid')

In [21]:
mex_t2g = mex.tower2grid('cities',1000)
mex_grids = mex.grids('cities', 1000)
mex_grids = mex_grids.merge(mex_t2g.groupby('grid')['gtid'].apply(lambda x: '<br/>'.join(x)).reset_index(), on='grid')

reading existing t2g file: data/mex_t2g_cities_1000m.csv
reading existing grids


## tower voronoi polygons for all cities

In [13]:
# plot the vor polygons in cities
some_map = folium.Map(location=[19.381495, -99.139095], zoom_start=8)

geojson_per_row(mex_cities.reset_index(), 'city', color='red', tip_cols=['city','population'],some_map=some_map)
geojson_per_row(mex_tvor.reset_index(), 'all_vor', color='blue', tip_cols=['gtid'],some_map=some_map)
geojson_per_row(mex_t2city, 'vor', color='yellow', tip_cols=['gtid','city','weight_x','weight_y','iarea_x','gtid_area_y'],some_map=some_map)
point_per_row(mex_tower[mex_tower.gtid.isin(mex_t2city.gtid)].reset_index(), 'tower', tip_cols=['gtid'],some_map=some_map)

folium.LayerControl().add_to(some_map)
some_map.save('maps/MexTwVorInCity.html')

## pick two cities to show how the grids look like

In [47]:
for c in ['Tijuana, Baja California', 'Valle de Mexico, Distrito Federal']:
    print('drawing for city', c)
    lon,lat = mex_cities.loc[c].geometry.centroid.coords[0]
    tmp_city = mex_cities.loc[[c]].reset_index()
    tmp_t2city = mex_t2city[mex_t2city.city==c]
    tmp_towers = mex_tower[mex_tower.gtid.isin(tmp_t2city.gtid)].reset_index()
    tmp_grids = mex_grids[mex_grids.city==c]
    
    some_map = folium.Map(location=[lat, lon], zoom_start=10)
    geojson_per_row(tmp_city, 'city', color='red', tip_cols=['city','population'],some_map=some_map)
    geojson_per_row(tmp_t2city, 'vor', color='blue', tip_cols=['gtid','city','weight'],some_map=some_map)
    geojson_per_row(tmp_grids, 'grids', color='yellow', tip_cols=['grid','city','gtid'],some_map=some_map)
    point_per_row(tmp_towers, 'tower', tip_cols=['gtid'],some_map=some_map)

    folium.LayerControl().add_to(some_map)
    some_map.save(f'maps/MexTwGridIn{c}.html')


drawing for city Tijuana, Baja California
drawing for city Valle de Mexico, Distrito Federal


# -----------Deprecated----------------

# Mexico tw2city

## all tower polygons

In [4]:
# get tower points
mex_tower = mex.tower()
# voronoi polygons across mexico
tw_vor_polys = lonlats2vorpolys(mex_tower.lonlat.tolist(),dataframe=True)
tw_vor_polys['gtid'] = mex_tower.gtid
tw_vor_polys.crs = mex_tower.crs
print(tw_vor_polys.gtid.value_counts().value_counts())
tw_vor_polys.set_index('gtid', inplace=True)

1    5731
Name: gtid, dtype: int64


In [6]:
# plot the vor polygons
# some_map = folium.Map(location=[19.381495, -99.139095], zoom_start=8)
# geojson_per_row(tw_vor_polys, 'vor', color='yellow', tip_cols=['gtid'],some_map=some_map)
# point_per_row(towers_shp, 'tower', tip_cols=['gtid'],some_map=some_map)

# folium.LayerControl().add_to(some_map)
# some_map.save('tmp/mexico_voronoi.html')

## tower polygons within cities(or other given regions)

In [7]:
mex_cities = mex.cities()

In [8]:
# intesect city polygons and tower polygons, 
# default weight=1 (the percentage of stats in that tower to be assigned to that city)
mex_t2c = gp.sjoin(tw_vor_polys, mex_cities)
mex_t2c['weight'] = 1

In [9]:
num_cities_per_tower = mex_t2c.groupby(level=0)['index_right'].count()
t_m_c = num_cities_per_tower[num_cities_per_tower>1]
t1c = num_cities_per_tower[num_cities_per_tower==1]

In [10]:
# for the tower polygons intersecting with multiple cities
# compute the area of each intersection
# divide the weight by the percentage of area
itxns = []
for gtid, row in mex_t2c.loc[t_m_c.index].iterrows():
    itxn = mex_cities.loc[row.index_right].geometry.intersection(tw_vor_polys.loc[gtid].geometry)
    itxns.append({'gtid': gtid, 'cname': row.index_right, 'geometry': itxn})
itxns = gp.GeoDataFrame(itxns)

itxns.crs = mex_cities.crs
itxns = itxns.to_crs(epsg=6362)
itxns['area'] = itxns.geometry.apply(lambda x: x.area)

itxns['weight'] = itxns.groupby('gtid').apply(lambda x: x.area/x.area.sum()).sort_index(level=1).values

In [11]:
columns = ['gtid','cname','weight']
tmp = mex_t2c.loc[t1c.index][['index_right','weight']].reset_index()
tmp.columns=columns
t_in_c = tmp.append(itxns[columns])
t_in_c['geometry'] = t_in_c.apply(lambda x: tw_vor_polys.loc[x.gtid].geometry.intersection(mex_cities.loc[x.cname].geometry), axis=1)
t_in_c = gp.GeoDataFrame(t_in_c)

In [12]:
# plot the vor polygons in cities
some_map = folium.Map(location=[19.381495, -99.139095], zoom_start=8)
geojson_per_row(mex_cities.reset_index(), 'city', color='red', tip_cols=['cname','population'],some_map=some_map)
geojson_per_row(t_in_c, 'vor', color='yellow', tip_cols=['gtid','cname','weight'],some_map=some_map)
point_per_row(mex_tower[mex_tower.gtid.isin(t_in_c.gtid)].reset_index(), 'tower', tip_cols=['gtid'],some_map=some_map)

folium.LayerControl().add_to(some_map)
some_map.save('data/tower_in_cities_mexico.html')

In [14]:
t_in_c.to_file('data/tower_in_cities_mexico.geojson', driver='GeoJSON')

In [18]:
t_in_c.groupby('cname')['gtid'].count().sort_values(ascending=False)

cname
Valle de Mexico, Distrito Federal    643
Monterrey, Nuevo León                326
Guadalajara, Jalisco                 269
Tijuana, Baja California             186
Mexicali, Baja California            112
Toluca de Lerdo, México               79
Cuernavaca, Morelos                   79
Chihuahua, Chihuahua                  74
Juárez, Chihuahua                     71
Acapulco de Juárez, Guerrero          68
Cancún, Quintana Roo                  62
Morelia, Michoacán de Ocampo          55
Villahermosa, Tabasco                 45
Nuevo Laredo, Tamaulipas              43
Culiacán Rosales, Sinaloa             41
Oaxaca de Juárez, Oaxaca              39
Name: gtid, dtype: int64